In [44]:
import pandas as pd
from fastai.vision.all import *

learn = load_learner("model.pkl")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [45]:
# Load the CSV
df = pd.read_csv("data/bodyfat_dataset.csv")

# OPTION A: Remove by index (e.g., delete row 10)
df = df.drop(index=176)

# OPTION B: Remove by condition (e.g., remove row where filename == '1_image_1.jpg')
# df = df[df['filename'] != '1_image_1.jpg']

# Save back to the same file
df.to_csv("data/bodyfat_datafat.csv", index=False)

In [46]:
def create_regression_csv(df, output_csv="data/image_labels.csv", label_col="meanPrediction", image_prefix="image_", output_dir="images"):
    # Ensure column names are stripped of whitespace
    df.columns = df.columns.str.strip()
    
    image_cols = [col for col in df.columns if col.startswith(image_prefix)]
    records = []

    for idx, row in df.iterrows():
        label = row[label_col]
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http"):
                ext = url.split('.')[-1].split('?')[0].lower()
                ext = ext if ext in ['jpg', 'jpeg', 'png', 'webp'] else 'jpg'
                filename = f"{idx}_{col}.{ext}"
                records.append({"filename": filename, "target": label})
    
    df_out = pd.DataFrame(records)
    df_out.to_csv(output_csv, index=False)
    print(f"Created {output_csv} with {len(df_out)} labeled images")
    return df_out

In [47]:
create_regression_csv(df)

Created data/image_labels.csv with 1596 labeled images


,filename,target
0,0_image_1.jpg,8.0
1,0_image_2.jpg,8.0
2,0_image_3.jpg,8.0
3,0_image_4.jpg,8.0
4,1_image_1.jpg,9.8
...,...,...
1591,801_image_1.jpg,12.0
1592,802_image_1.jpg,10.5
1593,803_image_1.jpg,15.0
1594,804_image_1.jpg,12.0


In [ ]:
from fastai.vision.all import PILImage, show_image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('data/image_labels_cropped.csv')  # adjust path as needed
df

In [48]:
from fastai.vision.all import PILImage, show_image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

# Load DataFrame
df = pd.read_csv('data/image_labels_cropped.csv')  # adjust path as needed
df
errors = []

# Create tqdm progress bar
pbar = tqdm(df.iterrows(), total=len(df), desc="Processing images", leave=True)

for i, row in pbar:
    try:
        img_path = f"images/{row['cropped_path']}"  # change if your image path differs
        img = PILImage.create(img_path)
        bf, _, probs = learn.predict(img)
        pred = float(probs[0])
        actual = float(row['target'])
        error = abs(pred - actual)

        errors.append({
            "img_path": img_path,
            "img": img,
            "actual": actual,
            "pred": pred,
            "error": error
        })

        # Optionally update progress bar postfix
        pbar.set_postfix({"last_error": f"{error:.2f}"})

    except Exception as e:
        pbar.write(f"Skipping {row['filename']} due to error: {e}")

# Sort and show top 10 worst predictions
top_errors = sorted(errors, key=lambda x: x['error'], reverse=True)[:10]

for idx, item in enumerate(top_errors):
    print(f"\n#{idx+1}")
    print(f"File: {item['img_path']}")
    print(f"Predicted: {item['pred']:.2f}")
    print(f"Actual: {item['actual']:.2f}")
    print(f"Error: {item['error']:.2f}")
    show_image(item['img'], title=f"#{idx+1}: Error {item['error']:.2f}")


Processing images:   6%|████▍                                                                        | 92/1596 [00:00<00:01, 916.39it/s]

Skipping 0_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipping 0_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipping 0_image_3.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipping 0_image_4.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/0_image_4.jpg'
Skipping 1_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/1_image_1.jpg'
Skipping 2_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipping 3_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/3_image_1.jpg'
Skipping 3_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/3_image_2.jpg'
Skipping 4_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/4_image_1.jpg'
Skipping 4_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipp

Processing images:   6%|████▍                                                                        | 92/1596 [00:00<00:01, 916.39it/s]

Skipping 69_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipping 69_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/69_image_2.jpg'
Skipping 69_image_3.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/69_image_3.jpg'
Skipping 69_image_4.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/69_image_4.jpg'
Skipping 70_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/70_image_1.jpg'
Skipping 70_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/70_image_2.jpg'
Skipping 70_image_3.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/70_image_3.jpg'
Skipping 71_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/71_image_1.jpg'
Skipping 71_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/71_image_2.jpg'


Processing images:  17%|████████████▊                                                               | 268/1596 [00:00<00:01, 705.23it/s]

...

Skipping 376_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/376_image_2.jpg'
Skipping 376_image_3.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/376_image_3.jpg'
Skipping 376_image_4.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/376_image_4.jpg'
Skipping 377_image_1.jpeg due to error: [Errno 2] No such file or directory: 'images/cropped_images/377_image_1.jpeg'
Skipping 378_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/378_image_1.jpg'
Skipping 378_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/378_image_2.jpg'
Skipping 378_image_3.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/378_image_3.jpg'
Skipping 379_image_1.jpeg due to error: [Errno 2] No such file or directory: 'images/cropped_images/379_image_1.jpeg'
Skipping 380_image_1.jpg due to error: [Errno 2] No such file or dir

Processing images:  56%|██████████████████████████████████████████▎                                 | 888/1596 [00:01<00:01, 677.73it/s]

Skipping 411_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/411_image_1.jpg'
Skipping 412_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipping 413_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/413_image_1.jpg'
Skipping 414_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/414_image_1.jpg'
Skipping 415_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/415_image_1.jpg'
Skipping 416_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/nan'
Skipping 417_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/417_image_1.jpg'
Skipping 418_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/418_image_1.jpg'
Skipping 419_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/419_image_1.jpg'
Skipping 4

Skipping 488_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/488_image_2.jpg'
Skipping 489_image_1.jpeg due to error: [Errno 2] No such file or directory: 'images/cropped_images/489_image_1.jpeg'
Skipping 490_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/490_image_1.jpg'
Skipping 490_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/490_image_2.jpg'
Skipping 490_image_3.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/490_image_3.jpg'
Skipping 491_image_1.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/491_image_1.jpg'
Skipping 491_image_2.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/491_image_2.jpg'
Skipping 491_image_3.jpg due to error: [Errno 2] No such file or directory: 'images/cropped_images/491_image_3.jpg'
Skipping 491_image_4.jpg due to error: [Errno 2] No such file or direc

Processing images:  61%|██████████████████████████████████████████████▏                             | 971/1596 [00:01<00:00, 646.11it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

